In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

In [3]:
np.random.seed(42)
dates = pd.date_range(start='2015-01-01', end='2024-01-01', freq='MS')
n = len(dates)

# --- BAGIAN A: MAKROEKONOMI (EXTERNAL) ---

In [4]:
base_cpi = np.random.normal(0.6, 0.2, n)
shock_cpi = np.zeros(n)
shock_cpi[-24:] = np.linspace(0, 2.2, 24) # Naik 2 tahun terakhir
cpi = base_cpi + shock_cpi + np.random.normal(0, 0.1, n)

# GDP Growth - Stabil, crash saat Covid (idx 60)

In [5]:
gdp = np.random.normal(1.8, 0.4, n)
gdp[60:66] = [-2.5, -4.0, -1.0, 0.5, 2.0, 3.5] # V-Shape Recovery

# SNB Policy Rate - Bereaksi terhadap Inflasi (Lagging)

In [6]:
rate = np.full(n, -0.75) # Start negatif
for i in range(3, n):
    if cpi[i-3] > 1.5: # Jika inflasi 3 bulan lalu tinggi
        rate[i] = rate[i-1] + 0.10 # Naikkan bunga
    else:
        rate[i] = max(-0.75, rate[i-1] - 0.05) # Turun/Tetap
rate = rate + np.random.normal(0, 0.02, n)

# Unemployment - Hukum Okun (Lawan arah GDP)

In [7]:
unemp = 2.8 - (gdp * 0.3) + np.random.normal(0, 0.1, n)
unemp = np.maximum(unemp, 1.5)

# EUR/CHF - Safe Haven (Rate naik -> Mata uang menguat/Angka turun)

In [8]:
eur_chf = 1.10 - (rate * 0.03) + np.random.normal(0, 0.01, n)

# --- BAGIAN B: INTERNAL ALPENBANK (INTERNAL) ---

# Loan Disbursement (Juta CHF) - Dipengaruhi GDP & Bunga
# GDP naik -> Kredit naik. Bunga naik -> Kredit turun.

In [9]:
seasonality = np.sin(np.linspace(0, 20, n)) * 4
loans = 60 + (gdp * 3) - (rate * 5) + seasonality + np.random.normal(0, 2, n)

# Default Rate (%) - Dipengaruhi Unemployment & Bunga

In [10]:
default_rate = 0.8 + (unemp * 0.2) + (rate * 0.15) + np.random.normal(0, 0.05, n)

In [11]:
df = pd.DataFrame({
    'Date': dates,
    'CPI_YoY': cpi.round(2),
    'SNB_Policy_Rate': rate.round(2),
    'GDP_Growth': gdp.round(2),
    'Unemployment': unemp.round(2),
    'EUR_CHF': eur_chf.round(4),
    'Loan_Disbursement': loans.round(1),
    'Default_Rate': default_rate.round(2)
})

df.set_index('Date', inplace=True)
df.to_csv('data/raw/alpenbank_macro_raw.csv')
print("Simpan Data")

Simpan Data
